1) snapme + diabetes food hub + gpt api -> make alternative diets
2) train/test split

In [122]:
import pandas as pd

mm = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data3_multimodal/version1/snapme_origin.csv")

In [123]:
mm.head(2)

,subject_id,before_caption,after_caption,recommendation,recommended_diet,before_image_path,after_image_path,general_info,food_identification_info,nutritional_info,food_classification_info,context_info,recommended_meal,instruction,input
0,SNAPME9001,coffee,coffee,### Analysis:\n\n**Strengths:**\n- Good source...,NaN,/data/jaesung/diabetes_rec/diellama/snapme/sna...,/data/jaesung/diabetes_rec/diellama/snapme/sna...,"{'snapme_study_day': 1, 'packaged_food': 0.0, ...","{'snapme_study_day': 1, 'packaged_food': 0.0, ...","{'PROT': 9.9324, 'TFAT': 6.0264, 'CARB': 27.60...","{'F_TOTAL': 0.0, 'F_JUICE': 0.0, 'V_STARCHY_TO...","Title: Citrus-Baked Pork Chops, Description: D...",Bountiful Harvest Vegetable Salad,Could you analyze the nutritional value of thi...,The image before the meal can be found at /dat...
1,SNAPME9001,orange,peeled orange,### Analysis:\n- **Strengths**:\n - Good sour...,- **Strengths**:,/data/jaesung/diabetes_rec/diellama/snapme/sna...,/data/jaesung/diabetes_rec/diellama/snapme/sna...,"{'snapme_study_day': 1, 'packaged_food': 0.0, ...","{'snapme_study_day': 1, 'packaged_food': 0.0, ...","{'PROT': 1.2314, 'TFAT': 0.1572, 'CARB': 15.39...","{'F_TOTAL': 0.7074, 'F_JUICE': 0.0, 'V_STARCHY...","Title: Just Peachy Yogurt and Granola Jar, Des...",Grilled Lime Chicken Fajitas,Could you provide an analysis of this meal and...,The pre-meal image is located at /data/jaesung...


In [124]:
import pandas as pd
from ast import literal_eval
import numpy as np

def extract_food_description(row):
    try:
        row = row.replace("nan", "None")
        data = literal_eval(row)
        return data.get('Food_Description', None) 
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Error parsing row: {row}, Error: {e}")
        return None 

mm['food_identification_info'] = mm['food_identification_info'].astype(str) 

mm2 = pd.DataFrame({
    'input': mm['food_identification_info'].apply(extract_food_description),
    'input_nutrition_facts': mm['nutritional_info'],
    'output': mm['recommended_meal']
})

In [125]:
mm2.head()

,input,input_nutrition_facts,output
0,"Coffee, Latte, flavored","{'PROT': 9.9324, 'TFAT': 6.0264, 'CARB': 27.60...",Bountiful Harvest Vegetable Salad
1,"Orange, raw","{'PROT': 1.2314, 'TFAT': 0.1572, 'CARB': 15.39...",Grilled Lime Chicken Fajitas
2,"Cheese, cottage, low fat","{'PROT': 17.71275, 'TFAT': 3.84765, 'CARB': 8....",Avocado Toast with Turkey Bacon and Tomato
3,"Blueberries, raw","{'PROT': 0.2738, 'TFAT': 0.1221, 'CARB': 5.361...",Air Fryer Spicy Green Beans
4,"Lettuce, arugula, raw","{'PROT': 0.3612, 'TFAT': 0.0924, 'CARB': 0.511...",Grilled Lime Chicken Fajitas


In [126]:
import pandas as pd
import ast
import re

NUTRITION_UNITS = {
    'PROT': 'g',   # 단백질
    'TFAT': 'g',   # 총 지방
    'CARB': 'g',   # 탄수화물
    'SUGR': 'g',   # 당
    'FIBE': 'g',   # 식이섬유
    'MAGN': 'mg',  # 마그네슘
    'POTA': 'mg',  # 칼륨
    'VB1': 'mg',   # 비타민 B1
    'VB6': 'mg',   # 비타민 B6
    'VB12': 'µg',  # 비타민 B12
    'SFAT': 'g'    # 포화 지방
}

def extract_context_info(context, title):
    try:
        entries = context.split("\n")
        for entry in entries:
            if f"Title: {title}," in entry:
                description = entry.split("Description:")[1].split(", Nutrition Facts:")[0].strip()
                nutrition_facts_str = entry.split("Nutrition Facts:")[1].split(", Ingredients:")[0].strip()
                nutrition_facts = ast.literal_eval(nutrition_facts_str)  # Nutrition Facts를 딕셔너리로 변환
                return description, nutrition_facts
    except Exception as e:
        print(f"Error processing title: {title}, Error: {e}")
    return None, None  

def add_units_to_nutrition_facts(nutrition_facts):
    """Nutrition Facts에 단위를 추가"""
    if not isinstance(nutrition_facts, dict):
        return nutrition_facts  
    with_units = {}
    for key, value in nutrition_facts.items():
        unit = NUTRITION_UNITS.get(key, '')  
        with_units[key] = f"{value} {unit}" if unit else value
    return with_units

output_descriptions = []
output_nutrition_facts_with_units = []

for i, row in mm.iterrows():
    title = row['recommended_meal']
    context = row['context_info']
    description, nutrition_facts = extract_context_info(context, title)
    output_descriptions.append(description)
    if nutrition_facts:
        try:
            # Nutrition Facts에 단위 추가
            nutrition_with_units = add_units_to_nutrition_facts(nutrition_facts)
            output_nutrition_facts_with_units.append(nutrition_with_units)
        except Exception as e:
            print(f"Error adding units to nutrition facts for {title}, Error: {e}")
            output_nutrition_facts_with_units.append(None)
    else:
        output_nutrition_facts_with_units.append(None)

mm2['input_nutrition_facts'] = mm2['input_nutrition_facts'].apply(lambda x: add_units_to_nutrition_facts(ast.literal_eval(x)))

mm2['output_description'] = output_descriptions
mm2['output_nutrition_facts'] = output_nutrition_facts_with_units


In [127]:
mm2.head(2)

,input,input_nutrition_facts,output,output_description,output_nutrition_facts
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Amount per serving': '145', 'Total Fat': '9g..."
1,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Amount per serving': '160', 'Total Fat': '4g..."


In [128]:
mm2['output_nutrition_facts'][1]

{'Amount per serving': '160',
 'Total Fat': '4g',
 'Cholesterol': '35mg',
 'Sodium': '370mg',
 'Total Carbohydrate': '22g',
 'Protein': '18g'}

In [129]:
import pandas as pd

dfh = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data2_daily_diets/diabetes_food_hub_new_nutri_facts.csv")

dfh.head(2)

,title,description,prep_time,cook_time,servings,steps,tags,nutrition_facts,ingredients
0,Raspberry Swirl Frozen Yogurt Bark,Raspberry Swirl Frozen Yogurt Bark: Dive into ...,10 min,4 hr,6 Servings,['Cover a freezer-safe tray with parchment pap...,"['Kid Friendly', 'Vegetarian', 'Dessert', 'Sna...","{'Servings': '6 Servings', 'Serving Size': '1 ...","[{'label': 'Plain Nonfat Greek yogurt', 'us_me..."
1,Maple-Pumpkin Spice Oatmeal Cookies,Description not found,10 min,25 min,14 Servings,['Preheat the oven to 350 degrees F. Line two ...,"['Kid Friendly', 'Vegetarian', 'Snacks', 'Glut...","{'Servings': '14 Servings', 'Serving Size': '1...","[{'label': 'old-fashioned rolled oats', 'us_me..."


In [130]:
dfh = dfh.drop_duplicates(subset='title')

In [131]:
dfh['nutrition_facts'][0]

"{'Servings': '6 Servings', 'Serving Size': '1 slice (4×4 inch square)', 'Amount per Serving': {'Calories': '70', 'Total Fat': {'Amount': '1g', 'Saturated Fat': '1g', 'Trans Fat': '0g'}, 'Cholesterol': '5mg', 'Sodium': '30mg', 'Total Carbohydrates': {'Amount': '19g', 'Dietary Fiber': '1g', 'Total Sugars': '4g', 'Added Sugars': '0g'}, 'Protein': '8g', 'Potassium': '140mg', 'Phosphorus': None}}"

In [132]:
dfh_tmp = dfh[['title', 'nutrition_facts']].drop_duplicates()

mm3 = pd.merge(mm2, dfh_tmp, left_on='output', right_on='title', how='left')

mm3.head(2)

,input,input_nutrition_facts,output,output_description,output_nutrition_facts,title,nutrition_facts
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Amount per serving': '145', 'Total Fat': '9g...",Bountiful Harvest Vegetable Salad,"{'Servings': '6 Servings', 'Serving Size': '2 ..."
1,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Amount per serving': '160', 'Total Fat': '4g...",Grilled Lime Chicken Fajitas,"{'Servings': '10 Servings', 'Serving Size': '1..."


In [133]:
mm3['nutrition_facts'].isna().sum()

mm3 = mm3[['input', 'input_nutrition_facts', 'output', 'output_description', 'nutrition_facts']]
mm3 = mm3.rename(columns={'nutrition_facts': 'output_nutrition_facts'})

mm3 = mm3.dropna(subset=['output_nutrition_facts'])

len(mm3)

3928

In [134]:
mm3.head(2)

,input,input_nutrition_facts,output,output_description,output_nutrition_facts
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ..."
1,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1..."


In [135]:
mm3['output_nutrition_facts'][0]

"{'Servings': '6 Servings', 'Serving Size': '2 cups', 'Amount per Serving': {'Calories': '145', 'Total Fat': {'Amount': '9g', 'Saturated Fat': '0.8g', 'Trans Fat': None}, 'Cholesterol': '0mg', 'Sodium': '45mg', 'Total Carbohydrates': {'Amount': '14g', 'Dietary Fiber': '4g', 'Total Sugars': '6g', 'Added Sugars': None}, 'Protein': '3g', 'Potassium': '415mg', 'Phosphorus': '85mg'}}"

In [137]:
import ast

# 필요한 키값 정의
REQUIRED_KEYS = [
    'Calories',              # 칼로리
    'Total Fat',             # 총 지방
    'Saturated Fat',         # 포화 지방
    'Trans Fat',             # 트랜스 지방
    'Cholesterol',           # 콜레스테롤
    'Sodium',                # 나트륨
    'Total Carbohydrates',   # 총 탄수화물
    'Dietary Fiber',         # 식이섬유
    'Total Sugars',          # 총당
    'Added Sugars',          # 추가당
    'Protein',               # 단백질
    'Potassium',             # 칼륨
    'Servings'               # 서빙 정보 추가
]

# Nutrition Facts 변환 함수
def extract_required_keys(nutrition_str):
    try:
        # 문자열을 딕셔너리로 변환
        nutrition_dict = ast.literal_eval(nutrition_str)
        
        # "Amount per Serving" 및 "Servings"에서 데이터 추출
        amount_per_serving = nutrition_dict.get('Amount per Serving', {})
        servings = nutrition_dict.get('Servings', 'Unknown')  # Servings 정보 추출
        extracted_data = {'Servings': servings}  # Servings 추가

        for key in REQUIRED_KEYS:
            if key == 'Servings':  # Servings는 이미 추가했으므로 스킵
                continue
            # 중첩된 딕셔너리인 경우 처리
            if key in ['Total Fat', 'Total Carbohydrates']:
                # 중첩된 Amount 및 세부항목 처리
                extracted_data[key] = amount_per_serving.get(key, {}).get('Amount', '0g')
                if key == 'Total Fat':
                    extracted_data['Saturated Fat'] = amount_per_serving.get(key, {}).get('Saturated Fat', '0g')
                    extracted_data['Trans Fat'] = amount_per_serving.get(key, {}).get('Trans Fat', '0g')
                if key == 'Total Carbohydrates':
                    extracted_data['Dietary Fiber'] = amount_per_serving.get(key, {}).get('Dietary Fiber', '0g')
                    extracted_data['Total Sugars'] = amount_per_serving.get(key, {}).get('Total Sugars', '0g')
                    extracted_data['Added Sugars'] = amount_per_serving.get(key, {}).get('Added Sugars', '0g')
            else:
                # 단일값 키 처리
                extracted_data[key] = amount_per_serving.get(key, '0')

        return extracted_data
    except Exception as e:
        print(f"Error processing nutrition facts: {nutrition_str}, Error: {e}")
        return None

# 변환 함수 적용
mm3['processed_output_nutrition_facts'] = mm3['output_nutrition_facts'].apply(extract_required_keys)

mm3.head(2)


,input,input_nutrition_facts,output,output_description,output_nutrition_facts,processed_output_nutrition_facts
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Servings': '6 Servings', 'Calories': '145', ..."
1,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Servings': '10 Servings', 'Calories': '160',..."


In [114]:
import pandas as pd
import ast
from difflib import SequenceMatcher
from tqdm import tqdm

# 숫자값 정규화 (float 형태로 변환)
def normalize_numbers(nutrition_data):
    try:
        if isinstance(nutrition_data, str):  # 문자열인 경우 딕셔너리로 변환
            nutrition_data = ast.literal_eval(nutrition_data)
        if isinstance(nutrition_data, dict):
            for key, value in nutrition_data.items():
                if isinstance(value, dict):
                    nutrition_data[key] = normalize_numbers(value)  # 재귀 호출
                elif isinstance(value, str) and any(char.isdigit() for char in value):  # 숫자 포함 문자열
                    value = value.replace("g", "").replace("mg", "").replace("µg", "").replace("VALUE", "")
                    try:
                        nutrition_data[key] = float(value)
                    except ValueError:
                        nutrition_data[key] = value
                elif isinstance(value, (int, float)):  # 숫자 값
                    nutrition_data[key] = float(value)
        return nutrition_data
    except Exception as e:
        print(f"Error normalizing numbers: {nutrition_data}, Error: {e}")
        return {}

# 문자열을 딕셔너리로 변환
def parse_nutrition_facts(nutrition_data):
    try:
        if isinstance(nutrition_data, dict):  # 이미 딕셔너리인 경우
            return nutrition_data
        elif isinstance(nutrition_data, str):  # 문자열인 경우 변환
            return ast.literal_eval(nutrition_data)
        else:  # 변환할 수 없는 경우 빈 딕셔너리 반환
            return {}
    except Exception as e:
        print(f"Error parsing nutrition facts: {nutrition_data}, Error: {e}")
        return {}

# 유사도 계산 함수 (숫자 정규화 후 비교)
def calculate_similarity(nutrition1, nutrition2):
    try:
        norm1 = normalize_numbers(nutrition1)
        norm2 = normalize_numbers(nutrition2)
        norm1_str = str(norm1)
        norm2_str = str(norm2)
        return SequenceMatcher(None, norm1_str, norm2_str).ratio()
    except Exception as e:
        print(f"Error calculating similarity: {e}")
        return 0

# 가장 유사한 두 개의 title을 찾아 반환
def find_two_most_similar_titles(output_nutrition_facts, dfh):
    similarities = []
    for _, row in dfh.iterrows():
        nutrition_facts = parse_nutrition_facts(row['nutrition_facts'])
        similarity = calculate_similarity(output_nutrition_facts, nutrition_facts)
        similarities.append((similarity, row))
    # 유사도를 기준으로 상위 두 개 선택
    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)[:2]
    results = [(row['title'], row['description'], row['nutrition_facts'], sim) for sim, row in similarities]
    # for result in results:
    #     print(f"Selected title: {result[0]} with similarity: {result[3]:.2f}")
    return results

# processed_output_nutrition_facts 기반으로 mm3 업데이트
def update_mm3_with_two_matches(mm3, dfh):
    updated_rows = []
    for _, row in tqdm(mm3.iterrows(), total=mm3.shape[0]):
        processed_facts = parse_nutrition_facts(row['processed_output_nutrition_facts'])
        similar_matches = find_two_most_similar_titles(processed_facts, dfh)
        
        updated_row = row.copy()
        if len(similar_matches) > 0:
            updated_row['output_1'] = similar_matches[0][0]
            updated_row['output_description_1'] = similar_matches[0][1]
            updated_row['output_nutrition_facts_1'] = similar_matches[0][2]
        if len(similar_matches) > 1:
            updated_row['output_2'] = similar_matches[1][0]
            updated_row['output_description_2'] = similar_matches[1][1]
            updated_row['output_nutrition_facts_2'] = similar_matches[1][2]
        updated_rows.append(updated_row)
    return pd.DataFrame(updated_rows)

# 데이터프레임 업데이트 실행
updated_mm3 = update_mm3_with_two_matches(mm3, dfh)
updated_mm3.head(2)


100%|██████████| 3928/3928 [40:29<00:00,  1.62it/s]


,input,input_nutrition_facts,output,output_description,output_nutrition_facts,processed_output_nutrition_facts,output_1,output_description_1,output_nutrition_facts_1,output_2,output_description_2,output_nutrition_facts_2
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Calories': 145.0, 'Total Fat': 9.0, 'Saturat...",Sesame Soy Salad Dressing,Description not found,"{'Servings': '16 Servings', 'Serving Size': '1...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ..."
1,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Calories': 160.0, 'Total Fat': 4.0, 'Saturat...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...",Caprese Kabobs,This easy appetizer is perfect for summer and ...,"{'Servings': '18 Servings', 'Serving Size': '1..."


In [138]:
# updated_mm3.to_csv("/data/jaesung/llm_for_diabetes/src/data/data3_multimodal/version1/tmp_updated_mm3.csv")

In [1]:
import pandas as pd

updated_mm3 = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data3_multimodal/version1/tmp_updated_mm3.csv")

updated_mm3.head(2)

,Unnamed: 0,input,input_nutrition_facts,output,output_description,output_nutrition_facts,processed_output_nutrition_facts,output_1,output_description_1,output_nutrition_facts_1,processed_output_nutrition_facts_1,output_2,output_description_2,output_nutrition_facts_2,processed_output_nutrition_facts_2
0,0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Calories': 145.0, 'Total Fat': 9.0, 'Saturat...",Sesame Soy Salad Dressing,Description not found,"{'Servings': '16 Servings', 'Serving Size': '1...","{'Calories': '50', 'Total Fat': '5g', 'Saturat...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Calories': '145', 'Total Fat': '9g', 'Satura..."
1,1,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Calories': 160.0, 'Total Fat': 4.0, 'Saturat...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Calories': '160', 'Total Fat': '4g', 'Satura...",Caprese Kabobs,This easy appetizer is perfect for summer and ...,"{'Servings': '18 Servings', 'Serving Size': '1...","{'Calories': '25', 'Total Fat': '2g', 'Saturat..."


In [2]:
updated_mm3[updated_mm3['input']=='Coffee, Latte, flavored']

,Unnamed: 0,input,input_nutrition_facts,output,output_description,output_nutrition_facts,processed_output_nutrition_facts,output_1,output_description_1,output_nutrition_facts_1,processed_output_nutrition_facts_1,output_2,output_description_2,output_nutrition_facts_2,processed_output_nutrition_facts_2
0,0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Calories': 145.0, 'Total Fat': 9.0, 'Saturat...",Sesame Soy Salad Dressing,Description not found,"{'Servings': '16 Servings', 'Serving Size': '1...","{'Calories': '50', 'Total Fat': '5g', 'Saturat...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Calories': '145', 'Total Fat': '9g', 'Satura..."
36,36,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Blueberry Lemon Yogurt Parfait,Take advantage of seasonal fresh berries in th...,"{'Servings': '4 Servings', 'Serving Size': '1 ...","{'Calories': 190.0, 'Total Fat': 4.0, 'Saturat...",Blueberry Lemon Yogurt Parfait,Take advantage of seasonal fresh berries in th...,"{'Servings': '4 Servings', 'Serving Size': '1 ...","{'Calories': '190', 'Total Fat': '4g', 'Satura...",Fruit & Nut Granola Bars,The ingredients in pre-packaged granola bars c...,"{'Servings': '18 Servings', 'Serving Size': '1...","{'Calories': '130', 'Total Fat': '8g', 'Satura..."


In [3]:
import ast

# 필요한 키값 정의
REQUIRED_KEYS = [
    'Calories',              # 칼로리
    'Total Fat',             # 총 지방
    'Saturated Fat',         # 포화 지방
    'Trans Fat',             # 트랜스 지방
    'Cholesterol',           # 콜레스테롤
    'Sodium',                # 나트륨
    'Total Carbohydrates',   # 총 탄수화물
    'Dietary Fiber',         # 식이섬유
    'Total Sugars',          # 총당
    'Added Sugars',          # 추가당
    'Protein',               # 단백질
    'Potassium',             # 칼륨
    'Servings'               # 서빙 정보 추가
]

# Nutrition Facts 변환 함수
def extract_required_keys(nutrition_str):
    try:
        # 문자열을 딕셔너리로 변환
        nutrition_dict = ast.literal_eval(nutrition_str)
        
        # "Amount per Serving" 및 "Servings"에서 데이터 추출
        amount_per_serving = nutrition_dict.get('Amount per Serving', {})
        servings = nutrition_dict.get('Servings', 'Unknown')  # Servings 정보 추출
        extracted_data = {'Servings': servings}  # Servings 추가

        for key in REQUIRED_KEYS:
            if key == 'Servings':  # Servings는 이미 추가했으므로 스킵
                continue
            # 중첩된 딕셔너리인 경우 처리
            if key in ['Total Fat', 'Total Carbohydrates']:
                # 중첩된 Amount 및 세부항목 처리
                extracted_data[key] = amount_per_serving.get(key, {}).get('Amount', '0g')
                if key == 'Total Fat':
                    extracted_data['Saturated Fat'] = amount_per_serving.get(key, {}).get('Saturated Fat', '0g')
                    extracted_data['Trans Fat'] = amount_per_serving.get(key, {}).get('Trans Fat', '0g')
                if key == 'Total Carbohydrates':
                    extracted_data['Dietary Fiber'] = amount_per_serving.get(key, {}).get('Dietary Fiber', '0g')
                    extracted_data['Total Sugars'] = amount_per_serving.get(key, {}).get('Total Sugars', '0g')
                    extracted_data['Added Sugars'] = amount_per_serving.get(key, {}).get('Added Sugars', '0g')
            else:
                # 단일값 키 처리
                extracted_data[key] = amount_per_serving.get(key, '0')

        return extracted_data
    except Exception as e:
        print(f"Error processing nutrition facts: {nutrition_str}, Error: {e}")
        return None

# 변환 함수 적용
updated_mm3['processed_output_nutrition_facts'] = updated_mm3['output_nutrition_facts'].apply(extract_required_keys)
updated_mm3['processed_output_nutrition_facts_1'] = updated_mm3['output_nutrition_facts_1'].apply(extract_required_keys)
updated_mm3['processed_output_nutrition_facts_2'] = updated_mm3['output_nutrition_facts_2'].apply(extract_required_keys)

# 필요한 컬럼만 남기기
updated_mm3 = updated_mm3[['input', 'input_nutrition_facts', 'output',
       'output_description', 'output_nutrition_facts',
       'processed_output_nutrition_facts', 'output_1', 'output_description_1',
       'output_nutrition_facts_1', 'processed_output_nutrition_facts_1', 'output_2', 'output_description_2',
       'output_nutrition_facts_2', 'processed_output_nutrition_facts_2']]

# 결과 확인
updated_mm3.head(2)

,input,input_nutrition_facts,output,output_description,output_nutrition_facts,processed_output_nutrition_facts,output_1,output_description_1,output_nutrition_facts_1,processed_output_nutrition_facts_1,output_2,output_description_2,output_nutrition_facts_2,processed_output_nutrition_facts_2
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Servings': '6 Servings', 'Calories': '145', ...",Sesame Soy Salad Dressing,Description not found,"{'Servings': '16 Servings', 'Serving Size': '1...","{'Servings': '16 Servings', 'Calories': '50', ...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Servings': '6 Servings', 'Calories': '145', ..."
1,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Servings': '10 Servings', 'Calories': '160',...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Servings': '10 Servings', 'Calories': '160',...",Caprese Kabobs,This easy appetizer is perfect for summer and ...,"{'Servings': '18 Servings', 'Serving Size': '1...","{'Servings': '18 Servings', 'Calories': '25', ..."


In [4]:
import pandas as pd

# 데이터프레임 확장 함수
def split_and_expand_rows(df):
    expanded_rows = []
    
    for _, row in df.iterrows():
        # 원래 행 추가
        original_row = row.copy()
        original_row['row_type'] = 'original'
        expanded_rows.append(original_row.to_dict())  # dict 형태로 추가

        # output_1 관련 데이터 추가
        new_row_1 = {
            'input': row['input'],
            'input_nutrition_facts': str(row['input_nutrition_facts']),
            'output': row['output_1'],
            'output_description': row['output_description_1'],
            'output_nutrition_facts': str(row['output_nutrition_facts_1']),
            'processed_output_nutrition_facts': str(row['processed_output_nutrition_facts_1']),
            'row_type': 'expanded_1'  # 구분용
        }
        expanded_rows.append(new_row_1)

        # output_2 관련 데이터 추가
        new_row_2 = {
            'input': row['input'],
            'input_nutrition_facts': str(row['input_nutrition_facts']),
            'output': row['output_2'],
            'output_description': row['output_description_2'],
            'output_nutrition_facts': str(row['output_nutrition_facts_2']),
            'processed_output_nutrition_facts': str(row['processed_output_nutrition_facts_2']),
            'row_type': 'expanded_2'  # 구분용
        }
        expanded_rows.append(new_row_2)

    # 새로운 데이터프레임 생성
    expanded_df = pd.DataFrame(expanded_rows)
    return expanded_df

# 함수 실행
updated_mm3 = split_and_expand_rows(updated_mm3)

# 결과 확인
updated_mm3.head(5)


,input,input_nutrition_facts,output,output_description,output_nutrition_facts,processed_output_nutrition_facts,output_1,output_description_1,output_nutrition_facts_1,processed_output_nutrition_facts_1,output_2,output_description_2,output_nutrition_facts_2,processed_output_nutrition_facts_2,row_type
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Servings': '6 Servings', 'Calories': '145', ...",Sesame Soy Salad Dressing,Description not found,"{'Servings': '16 Servings', 'Serving Size': '1...","{'Servings': '16 Servings', 'Calories': '50', ...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Servings': '6 Servings', 'Calories': '145', ...",original
1,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Sesame Soy Salad Dressing,Description not found,"{'Servings': '16 Servings', 'Serving Size': '1...","{'Servings': '16 Servings', 'Calories': '50', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,expanded_1
2,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Servings': '6 Servings', 'Calories': '145', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,expanded_2
3,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Servings': '10 Servings', 'Calories': '160',...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Servings': '10 Servings', 'Calories': '160',...",Caprese Kabobs,This easy appetizer is perfect for summer and ...,"{'Servings': '18 Servings', 'Serving Size': '1...","{'Servings': '18 Servings', 'Calories': '25', ...",original
4,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...","{'Servings': '10 Servings', 'Calories': '160',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,expanded_1


In [5]:
updated_mm3 = updated_mm3[['input', 'input_nutrition_facts', 'output', 'output_description', 'output_nutrition_facts', 'processed_output_nutrition_facts']]

updated_mm3.head(2)

,input,input_nutrition_facts,output,output_description,output_nutrition_facts,processed_output_nutrition_facts
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...","{'Servings': '6 Servings', 'Calories': '145', ..."
1,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Sesame Soy Salad Dressing,Description not found,"{'Servings': '16 Servings', 'Serving Size': '1...","{'Servings': '16 Servings', 'Calories': '50', ..."


In [6]:
# 데이터 바꾸기 전까지만 실행
updated_mm3 = updated_mm3[updated_mm3['output_description']!='Description not found']

In [ ]:
import openai
import pandas as pd
import json
from tqdm import tqdm

from dotenv import load_dotenv
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_reason(input_food, input_facts, output_food, output_desc, output_facts):
    try:
        prompt = (
            f"The consumed food '{input_food}' has certain nutritional deficiencies as detailed in: {input_facts}. "
            f"The recommended food '{output_food}' can help address these deficiencies based on the following details: "
            f"Description: {output_desc}, Nutrition Information: {output_facts}. "
            f"Explain briefly why '{output_food}' complements '{input_food}' without mentioning exact nutritional values. "
            f"If there are any notable drawbacks of '{output_food}', mention them as cautionary points. "
            f"Write a factual and concise explanation in 1-2 sentences."
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=80,
            temperature=0.7
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error: {e}"

def save_to_json(file_path, data):
    try:
        try:
            with open(file_path, "r") as f:
                existing_data = json.load(f)
        except FileNotFoundError:
            existing_data = []  

        existing_data.append(data)

        with open(file_path, "w") as f:
            json.dump(existing_data, f, indent=4)
    except Exception as e:
        print(f"Error saving to JSON: {e}")

file_path = "snapme_origin.json"

def process_and_save():
    for idx, row in tqdm(updated_mm3.iterrows(), total=updated_mm3.shape[0], desc="Processing rows"):
        reason = generate_reason(
            row['input'], row['input_nutrition_facts'],
            row['output'], row['output_description'],
            row['output_nutrition_facts']
        )

        result = {
            "input": row['input'],
            "input_nutrition_facts": row['input_nutrition_facts'],
            "output": row['output'],
            "output_description": row['output_description'],
            "output_nutrition_facts": row['output_nutrition_facts'],
            "reason": reason
        }

        save_to_json(file_path, result)

process_and_save()

print(f"Reason generation complete. Results saved to {file_path}.")


Processing rows: 100%|██████████| 8611/8611 [2:33:45<00:00,  1.07s/it]  

Reason generation complete. Results saved to snapme_origin_v1.json.


In [9]:
import json

# file_path = "/data/jaesung/llm_for_diabetes/src/data/data3_multimodal/snapme_origin_v1.json"
file_path = "/data/jaesung/llm_for_diabetes/src/data/data3_multimodal/version1/snapme_origin.json"
with open(file_path, 'r', encoding='utf-8') as f:
    tmp = json.load(f)

mm4 = pd.DataFrame(tmp)

In [10]:
mm4.head()

,input,input_nutrition_facts,output,output_description,output_nutrition_facts,reason
0,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...",The Bountiful Harvest Vegetable Salad compleme...
1,"Coffee, Latte, flavored","{'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...",Bountiful Harvest Vegetable Salad,This is a wonderful way to use fresh produce f...,"{'Servings': '6 Servings', 'Serving Size': '2 ...",Bountiful Harvest Vegetable Salad complements ...
2,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...",'Grilled Lime Chicken Fajitas' complements 'Or...
3,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Grilled Lime Chicken Fajitas,Chicken fajitas are one of the best choices wh...,"{'Servings': '10 Servings', 'Serving Size': '1...",'Grilled Lime Chicken Fajitas' complement 'Ora...
4,"Orange, raw","{'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...",Caprese Kabobs,This easy appetizer is perfect for summer and ...,"{'Servings': '18 Servings', 'Serving Size': '1...","'Caprese Kabobs' complement 'Orange, raw' as t..."


In [12]:
len(mm4)

8611

In [13]:
from sklearn.model_selection import train_test_split

# 1. 고유한 input 값을 기준으로 그룹 생성
input_groups = mm4.groupby('input')

# 2. 고유한 input 값을 리스트로 추출
unique_inputs = list(input_groups.groups.keys())

# 3. train-test split 수행 (unique input 값 기준)
train_inputs, test_inputs = train_test_split(unique_inputs, test_size=0.2, random_state=42)

# 4. 각 그룹에 속한 행을 train/test로 분리
train_df = mm4[mm4['input'].isin(train_inputs)].copy()
test_df = mm4[mm4['input'].isin(test_inputs)].copy()

# 결과 출력
print(f"Train 데이터 개수: {len(train_df)}")
print(f"Test 데이터 개수: {len(test_df)}")

# Train/Test 데이터 확인
print("Train Data Sample:")
print(train_df.head())

print("\nTest Data Sample:")
print(test_df.head())

Train 데이터 개수: 7046
Test 데이터 개수: 1565
Train Data Sample:
                     input                              input_nutrition_facts  \
0  Coffee, Latte, flavored  {'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...   
1  Coffee, Latte, flavored  {'PROT': '9.9324 g', 'TFAT': '6.0264 g', 'CARB...   
2              Orange, raw  {'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...   
3              Orange, raw  {'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...   
4              Orange, raw  {'PROT': '1.2314 g', 'TFAT': '0.1572 g', 'CARB...   

                              output  \
0  Bountiful Harvest Vegetable Salad   
1  Bountiful Harvest Vegetable Salad   
2       Grilled Lime Chicken Fajitas   
3       Grilled Lime Chicken Fajitas   
4                     Caprese Kabobs   

                                  output_description  \
0  This is a wonderful way to use fresh produce f...   
1  This is a wonderful way to use fresh produce f...   
2  Chicken fajitas are one of the best choices w

In [14]:
import pandas as pd
import json

def generate_json_from_df(df):
    results = []
    for _, row in df.iterrows():
        result = {
            "instruction": "Based on the previous meal, suggest the next meal to maintain a balanced diet.",
            "input": row["input"],
            "output": f"{row['output']} is recommended. The reason is {row['reason']}"
        }
        results.append(result)
    return results

json_results = generate_json_from_df(train_df)

with open("snapme_train_instruction_dataset.json", "w", encoding="utf-8") as f:
    json.dump(json_results, f, ensure_ascii=False, indent=4)

print(json.dumps(json_results[:2], ensure_ascii=False, indent=4))


[
    {
        "instruction": "Based on the previous meal, suggest the next meal to maintain a balanced diet.",
        "input": "Coffee, Latte, flavored",
        "output": "Bountiful Harvest Vegetable Salad is recommended. The reason is The Bountiful Harvest Vegetable Salad complements the Coffee, Latte, flavored as it provides a good source of fiber, vitamins, and minerals that are lacking in the latte. However, individuals should be cautious of any added dressings or toppings that may increase the calorie and fat content of the salad."
    },
    {
        "instruction": "Based on the previous meal, suggest the next meal to maintain a balanced diet.",
        "input": "Coffee, Latte, flavored",
        "output": "Bountiful Harvest Vegetable Salad is recommended. The reason is Bountiful Harvest Vegetable Salad complements Coffee, Latte, flavored by providing a balance of nutrients such as fiber, vitamins, and minerals that are lacking in the latte. However, the salad may not provid

In [15]:
import pandas as pd
import json

def generate_json_from_df(df):
    results = []
    for _, row in df.iterrows():
        result = {
            "instruction": "Based on the previous meal, suggest the next meal to maintain a balanced diet.",
            "input": row["input"],
            "output": f"{row['output']} is recommended. The reason is {row['reason']}"
        }
        results.append(result)
    return results

json_results = generate_json_from_df(test_df)

with open("snapme_test_instruction_dataset.json", "w", encoding="utf-8") as f:
    json.dump(json_results, f, ensure_ascii=False, indent=4)

print(json.dumps(json_results[:2], ensure_ascii=False, indent=4))


[
    {
        "instruction": "Based on the previous meal, suggest the next meal to maintain a balanced diet.",
        "input": "Mixed salad greens, raw",
        "output": "Greek Salmon and Veggie Packets is recommended. The reason is 'Greek Salmon and Veggie Packets' complements 'Mixed salad greens, raw' by providing a balance of protein, healthy fats, and essential nutrients that are lacking in the salad. However, caution should be taken with the sodium content in the dish, as it may be high for individuals with certain dietary restrictions or health conditions."
    },
    {
        "instruction": "Based on the previous meal, suggest the next meal to maintain a balanced diet.",
        "input": "Mixed salad greens, raw",
        "output": "Greek Salmon and Veggie Packets is recommended. The reason is 'Greek Salmon and Veggie Packets' complements 'Mixed salad greens, raw' by providing a good source of protein, omega-3 fatty acids, and potassium, which are lacking in the salad. How